In [10]:
import os
import json
import pandas
import traceback
from dotenv import load_dotenv

In [12]:
load_dotenv()
open_api_key=os.getenv('openai_api_key')


In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [13]:
llm=ChatOpenAI(api_key=open_api_key,model="gpt-3.5-turbo",temperature=0.5)